In [2]:
from openvino.inference_engine import IECore
import numpy as np
import cv2

ModuleNotFoundError: No module named 'openvino'

In [ ]:
def toTensor(data):
    '''Change the data range to [0.0, 1.0]
    Return [B, C, H, W]
    '''
    assert data.ndim == 2 or data.ndim == 3
    if data.ndim == 2:
        # depth
        data = data[..., np.newaxis]
        mask = data > 10000
        data[mask] = 0
        data = data / 10000.
    elif data.ndim == 3:
        # image
        data = data / 255.
    data = np.transpose(data, (2, 0, 1))
    data = data[np.newaxis, ...]
    return data

def normalize(data, mean, std):
    assert data.ndim == 4
    assert data.shape[1] == len(mean) == len(std)
    batch_size = data.shape[0]
    mean, std = np.asarray(mean), np.asarray(std)
    mean = mean[..., np.newaxis, np.newaxis]
    std = std[..., np.newaxis, np.newaxis]
    for i in range(batch_size):
        data[i, ...] = (data[i, ...] - mean) / std
    return data

In [ ]:
ie = IECore()
xml_file = './workspace/openvino/model1/FP32/DFA_model.xml'
bin_file = './workspace/openvino/model1/FP32/DFA_model.bin'

i_file = './workspace/openvino/test/rs02_1564022345_98.jpg'
d_file = './workspace/openvino/test/rs02_1564022345_98.pgm'

i = cv2.imread(i_file)
d = cv2.imread(d_file, cv2.IMREAD_ANYDEPTH)

data = toTensor(d)
data = normalize(data, [0.1864497], [0.07711394])

img_data_ = toTensor(i)
img_data = normalize(img_data_, [0.368, 0.393, 0.404], [0.286, 0.290, 0.296])

inputs = np.concatenate((img_data, data), axis=1)

net = ie.read_network(model=xml_file, weights=bin_file)
inputLayer = next(iter(net.inputs))

exec_net = ie.load_network(network=net, device_name='GPU')
results = exec_net.infer(inputs={inputLayer: inputs})

In [ ]:
import time
t0 = time.time()
for i in range(99):
    results = exec_net.infer(inputs={inputLayer: inputs})
t1 = time.time()

t = t1 - t0
1. / (t/99.)